In [1]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Task 1: Install scikit-learn and load a sample dataset

# If running in a new environment, uncomment the next line:

from sklearn.datasets import load_boston
import pandas as pd

# Load the Boston housing dataset
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
print("Sample of the dataset:")
print(X.head())

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

In [ ]:
# Task 2: Initialize Isolation Forest and fit to the Boston dataset

from sklearn.ensemble import IsolationForest

# Initialize Isolation Forest
iso_forest = IsolationForest(random_state=42)
# Fit the model
iso_forest.fit(X)
print("Isolation Forest model fitted.")

NameError: name 'X' is not defined

In [ ]:
# Task 3: Predict anomalies and display anomaly counts

# Predict anomalies (-1 for anomaly, 1 for normal)
anomaly_labels = iso_forest.predict(X)
# Add to DataFrame for convenience
X['anomaly'] = anomaly_labels

# Display anomaly counts
print("Anomaly counts:")
print(X['anomaly'].value_counts())

NameError: name 'X' is not defined

In [ ]:
# Task 4: Visualize the results with a scatter plot

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.scatter(X['RM'], X['LSTAT'], c=X['anomaly'], cmap='coolwarm', edgecolor='k')
plt.xlabel('Average number of rooms (RM)')
plt.ylabel('% lower status of the population (LSTAT)')
plt.title('Isolation Forest Anomaly Detection')
plt.show()

NameError: name 'X' is not defined

<Figure size 800x500 with 0 Axes>

In [ ]:
# Task 5: Experiment with different contamination levels

for contamination in [0.01, 0.05, 0.1, 0.2]:
    iso = IsolationForest(contamination=contamination, random_state=42)
    iso.fit(X.drop('anomaly', axis=1))
    preds = iso.predict(X.drop('anomaly', axis=1))
    print(f"Contamination={contamination}: Anomalies detected = {(preds == -1).sum()}")

NameError: name 'X' is not defined